# Datentransformation Previous Application

Ziel dieser Datenaufbereitung ist es alle Variablen so in einen Datensatz zu überführen.

In [22]:
from pathlib import Path
from scipy import stats

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

np.set_printoptions(suppress=True)

pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.options.display.max_colwidth = None

from IPython.display import display, Markdown

In [23]:
path1 = Path(r"A:\Workspace\Python\Masterarbeit\Kaggle Home Credit Datensatz")
path2 = Path(r"C:\Users\rober\Documents\Workspace\Python\Masterarbeit\Kaggle Home Credit Datensatz")

if path1.is_dir():
    DATASET_DIR = path1
else:
    DATASET_DIR = path2

In [24]:
app_train = pd.read_csv(DATASET_DIR / "application_train.csv")
previous_application = pd.read_csv(DATASET_DIR / "previous_application.csv")
des = pd.read_csv(DATASET_DIR / "HomeCredit_columns_description.csv", encoding="latin", index_col=0)

In [25]:
todo = list(previous_application.columns)

In [26]:
previous_application.head()

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_TYPE_SUITE,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,SELLERPLACE_AREA,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,2030495,271877,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,Y,1,0.0,0.182832,0.867336,XAP,Approved,-73,Cash through the bank,XAP,NaN,Repeater,Mobile,POS,XNA,Country-wide,35,Connectivity,12.0,middle,POS mobile with interest,365243.0,-42.0,300.0,-42.0,-37.0,0.0
1,2802425,108129,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,Y,1,NaN,NaN,NaN,XNA,Approved,-164,XNA,XAP,Unaccompanied,Repeater,XNA,Cash,x-sell,Contact center,-1,XNA,36.0,low_action,Cash X-Sell: low,365243.0,-134.0,916.0,365243.0,365243.0,1.0
2,2523466,122040,Cash loans,15060.735,112500.0,136444.5,NaN,112500.0,TUESDAY,11,Y,1,NaN,NaN,NaN,XNA,Approved,-301,Cash through the bank,XAP,"Spouse, partner",Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,12.0,high,Cash X-Sell: high,365243.0,-271.0,59.0,365243.0,365243.0,1.0
3,2819243,176158,Cash loans,47041.335,450000.0,470790.0,NaN,450000.0,MONDAY,7,Y,1,NaN,NaN,NaN,XNA,Approved,-512,Cash through the bank,XAP,NaN,Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,12.0,middle,Cash X-Sell: middle,365243.0,-482.0,-152.0,-182.0,-177.0,1.0
4,1784265,202054,Cash loans,31924.395,337500.0,404055.0,NaN,337500.0,THURSDAY,9,Y,1,NaN,NaN,NaN,Repairs,Refused,-781,Cash through the bank,HC,NaN,Repeater,XNA,Cash,walk-in,Credit and cash offices,-1,XNA,24.0,high,Cash Street: high,NaN,NaN,NaN,NaN,NaN,NaN


#### Keine Datenaufbereitung für Primär- und Fremdschlüssel

In [27]:
keys = ["SK_ID_CURR", "SK_ID_PREV"]

In [28]:
todo = [element for element in todo if element not in keys]

#### Datenaufbereitung metrische Daten

In [29]:
m_heads = [element for element in todo if previous_application[element].dtype.name != "object"]

##### Gruppierung

In [30]:
quants = [0, 0.25, 0.5, 0.75, 1]

In [31]:
for index, head in enumerate(m_heads):
    
    df = previous_application[["SK_ID_CURR", head]]
    df = df.groupby(by = ["SK_ID_CURR"]).quantile(quants)
    df = df.unstack(level=-1)
    
    try:
        mets = mets.join(df , on = "SK_ID_CURR")
    except (ValueError, NameError):
        mets = df

In [32]:
# Löschen der metrischen Daten aus der TODO Liste
todo = [element for element in todo if element not in m_heads]

In [33]:
mets.head()

AMT_ANNUITY                                               \
                  0.00        0.25       0.50       0.75       1.00   
SK_ID_CURR                                                            
100001        3951.000   3951.0000   3951.000   3951.000   3951.000   
100002        9251.775   9251.7750   9251.775   9251.775   9251.775   
100003        6737.310  35652.4875  64567.665  81462.330  98356.995   
100004        5357.250   5357.2500   5357.250   5357.250   5357.250   
100005        4813.200   4813.2000   4813.200   4813.200   4813.200   

           AMT_APPLICATION                                               \
                      0.00        0.25       0.50        0.75      1.00   
SK_ID_CURR                                                                
100001             24835.5   24835.500   24835.50   24835.500   24835.5   
100002            179055.0  179055.000  179055.00  179055.000  179055.0   
100003             68809.5  203154.750  337500.00  618750.000  900000.0   
100004             24282.0   24282.000   24282.00   24282.000   24282.0   
100005                 0.0   11154.375   22308.75   33463.125   44617.5   

           AMT_CREDIT                                                \
                 0.00        0.25       0.50        0.75       1.00   
SK_ID_CURR                                                            
100001        23787.0   23787.000   23787.00   23787.000    23787.0   
100002       179055.0  179055.000  179055.00  179055.000   179055.0   
100003        68053.5  208345.500  348637.50  692259.750  1035882.0   
100004        20106.0   20106.000   20106.00   20106.000    20106.0   
100005            0.0   10038.375   20076.75   30115.125    40153.5   

           AMT_DOWN_PAYMENT                                   AMT_GOODS_PRICE  \
                       0.00     0.25    0.50     0.75    1.00            0.00   
SK_ID_CURR                                                                      
100001               2520.0  2520.00  2520.0  2520.00  2520.0         24835.5   
100002                  0.0     0.00     0.0     0.00     0.0        179055.0   
100003                  0.0  1721.25  3442.5  5163.75  6885.0         68809.5   
100004               4860.0  4860.00  4860.0  4860.00  4860.0         24282.0   
100005               4464.0  4464.00  4464.0  4464.00  4464.0         44617.5   

                                                    HOUR_APPR_PROCESS_START  \
                 0.25      0.50      0.75      1.00                    0.00   
SK_ID_CURR                                                                    
100001       24835.50   24835.5   24835.5   24835.5                    13.0   
100002      179055.00  179055.0  179055.0  179055.0                     9.0   
100003      203154.75  337500.0  618750.0  900000.0                    12.0   
100004       24282.00   24282.0   24282.0   24282.0                     5.0   
100005       44617.50   44617.5   44617.5   44617.5                    10.0   

                                     NFLAG_LAST_APPL_IN_DAY                 \
             0.25  0.50   0.75  1.00                   0.00 0.25 0.50 0.75   
SK_ID_CURR                                                                   
100001      13.00  13.0  13.00  13.0                    1.0  1.0  1.0  1.0   
100002       9.00   9.0   9.00   9.0                    1.0  1.0  1.0  1.0   
100003      13.50  15.0  16.00  17.0                    1.0  1.0  1.0  1.0   
100004       5.00   5.0   5.00   5.0                    1.0  1.0  1.0  1.0   
100005      10.25  10.5  10.75  11.0                    1.0  1.0  1.0  1.0   

                RATE_DOWN_PAYMENT                                          \
           1.00              0.00      0.25      0.50      0.75      1.00   
SK_ID_CURR                                                                  
100001      1.0          0.104326  0.104326  0.104326  0.104326  0.104326   
100002      1.0          0.000000  0.000000  0.000000  0.

#### Datenaufbereitung kategorische Daten

In [34]:
cat_heads = [element for element in todo if previous_application[element].dtype.name == "object"]

In [35]:
for index, head in enumerate(cat_heads):
    
    df = previous_application[["SK_ID_CURR", head]]
    df = df.groupby(["SK_ID_CURR", head]).size()
    df = df.unstack()
    
    try:
        cats = cats.join(a , on="SK_ID_CURR")
    except (ValueError, NameError):
        cats = df

In [36]:
# Löschen der metrischen Daten aus der TODO Liste
todo = [element for element in todo if element not in cat_heads]

In [37]:
cats.head()

PRODUCT_COMBINATION,Card Street,Card X-Sell,Cash,Cash Street: high,Cash Street: low,Cash Street: middle,Cash X-Sell: high,Cash X-Sell: low,Cash X-Sell: middle,POS household with interest,POS household without interest,POS industry with interest,POS industry without interest,POS mobile with interest,POS mobile without interest,POS other with interest,POS others without interest
SK_ID_CURR,,,,,,,,,,,,,,,,,
100001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
100002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
100003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN
100004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
100005,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN


#### Zusammenführung metrischer und kategorischer Daten

In [38]:
previous_application = mets.join(cats, on = "SK_ID_CURR")

c:\users\rober\documents\workspace\python\masterarbeit\venv\lib\site-packages\pandas\core\reshape\merge.py:643: UserWarning: merging between different levels can give an unintended result (2 levels on the left,1 on the right)
  warnings.warn(msg, UserWarning)


In [39]:
previous_application.head()

,"(AMT_ANNUITY, 0.0)","(AMT_ANNUITY, 0.25)","(AMT_ANNUITY, 0.5)","(AMT_ANNUITY, 0.75)","(AMT_ANNUITY, 1.0)","(AMT_APPLICATION, 0.0)","(AMT_APPLICATION, 0.25)","(AMT_APPLICATION, 0.5)","(AMT_APPLICATION, 0.75)","(AMT_APPLICATION, 1.0)","(AMT_CREDIT, 0.0)","(AMT_CREDIT, 0.25)","(AMT_CREDIT, 0.5)","(AMT_CREDIT, 0.75)","(AMT_CREDIT, 1.0)","(AMT_DOWN_PAYMENT, 0.0)","(AMT_DOWN_PAYMENT, 0.25)","(AMT_DOWN_PAYMENT, 0.5)","(AMT_DOWN_PAYMENT, 0.75)","(AMT_DOWN_PAYMENT, 1.0)","(AMT_GOODS_PRICE, 0.0)","(AMT_GOODS_PRICE, 0.25)","(AMT_GOODS_PRICE, 0.5)","(AMT_GOODS_PRICE, 0.75)","(AMT_GOODS_PRICE, 1.0)","(HOUR_APPR_PROCESS_START, 0.0)","(HOUR_APPR_PROCESS_START, 0.25)","(HOUR_APPR_PROCESS_START, 0.5)","(HOUR_APPR_PROCESS_START, 0.75)","(HOUR_APPR_PROCESS_START, 1.0)","(NFLAG_LAST_APPL_IN_DAY, 0.0)","(NFLAG_LAST_APPL_IN_DAY, 0.25)","(NFLAG_LAST_APPL_IN_DAY, 0.5)","(NFLAG_LAST_APPL_IN_DAY, 0.75)","(NFLAG_LAST_APPL_IN_DAY, 1.0)","(RATE_DOWN_PAYMENT, 0.0)","(RATE_DOWN_PAYMENT, 0.25)","(RATE_DOWN_PAYMENT, 0.5)","(RATE_DOWN_PAYMENT, 0.75)","(RATE_DOWN_PAYMENT, 1.0)","(RATE_INTEREST_PRIMARY, 0.0)","(RATE_INTEREST_PRIMARY, 0.25)","(RATE_INTEREST_PRIMARY, 0.5)","(RATE_INTEREST_PRIMARY, 0.75)","(RATE_INTEREST_PRIMARY, 1.0)","(RATE_INTEREST_PRIVILEGED, 0.0)","(RATE_INTEREST_PRIVILEGED, 0.25)","(RATE_INTEREST_PRIVILEGED, 0.5)","(RATE_INTEREST_PRIVILEGED, 0.75)","(RATE_INTEREST_PRIVILEGED, 1.0)","(DAYS_DECISION, 0.0)","(DAYS_DECISION, 0.25)","(DAYS_DECISION, 0.5)","(DAYS_DECISION, 0.75)","(DAYS_DECISION, 1.0)","(SELLERPLACE_AREA, 0.0)","(SELLERPLACE_AREA, 0.25)","(SELLERPLACE_AREA, 0.5)","(SELLERPLACE_AREA, 0.75)","(SELLERPLACE_AREA, 1.0)","(CNT_PAYMENT, 0.0)","(CNT_PAYMENT, 0.25)","(CNT_PAYMENT, 0.5)","(CNT_PAYMENT, 0.75)","(CNT_PAYMENT, 1.0)","(DAYS_FIRST_DRAWING, 0.0)","(DAYS_FIRST_DRAWING, 0.25)","(DAYS_FIRST_DRAWING, 0.5)","(DAYS_FIRST_DRAWING, 0.75)","(DAYS_FIRST_DRAWING, 1.0)","(DAYS_FIRST_DUE, 0.0)","(DAYS_FIRST_DUE, 0.25)","(DAYS_FIRST_DUE, 0.5)","(DAYS_FIRST_DUE, 0.75)","(DAYS_FIRST_DUE, 1.0)","(DAYS_LAST_DUE_1ST_VERSION, 0.0)","(DAYS_LAST_DUE_1ST_VERSION, 0.25)","(DAYS_LAST_DUE_1ST_VERSION, 0.5)","(DAYS_LAST_DUE_1ST_VERSION, 0.75)","(DAYS_LAST_DUE_1ST_VERSION, 1.0)","(DAYS_LAST_DUE, 0.0)","(DAYS_LAST_DUE, 0.25)","(DAYS_LAST_DUE, 0.5)","(DAYS_LAST_DUE, 0.75)","(DAYS_LAST_DUE, 1.0)","(DAYS_TERMINATION, 0.0)","(DAYS_TERMINATION, 0.25)","(DAYS_TERMINATION, 0.5)","(DAYS_TERMINATION, 0.75)","(DAYS_TERMINATION, 1.0)","(NFLAG_INSURED_ON_APPROVAL, 0.0)","(NFLAG_INSURED_ON_APPROVAL, 0.25)","(NFLAG_INSURED_ON_APPROVAL, 0.5)","(NFLAG_INSURED_ON_APPROVAL, 0.75)","(NFLAG_INSURED_ON_APPROVAL, 1.0)",Card Street,Card X-Sell,Cash,Cash Street: high,Cash Street: low,Cash Street: middle,Cash X-Sell: high,Cash X-Sell: low,Cash X-Sell: middle,POS household with interest,POS household without interest,POS industry with interest,POS industry without interest,POS mobile with interest,POS mobile without interest,POS other with interest,POS others without interest
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
100001,3951.000,3951.0000,3951.000,3951.000,3951.000,24835.5,24835.500,24835.50,24835.500,24835.5,23787.0,23787.000,23787.00,23787.000,23787.0,2520.0,2520.00,2520.0,2520.00,2520.0,24835.5,24835.50,24835.5,24835.5,24835.5,13.0,13.00,13.0,13.00,13.0,1.0,1.0,1.0,1.0,1.0,0.104326,0.104326,0.104326,0.104326,0.104326,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1740.0,-1740.0,-1740.0,-1740.0,-1740.0,23.0,23.0,23.0,23.0,23.0,8.0,8.0,8.0,8.0,8.0,365243.0,365243.0,365243.0,365243.0,365243.0,-1709.0,-1709.0,-1709.0,-1709.0,-1709.0,-1499.0,-1499.0,-1499.0,-1499.0,-1499.0,-1619.0,-1619.0,-1619.0,-1619.0,-1619.0,-1612.0,-1612.0,-1612.0,-1612.0,-1612.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
100002,9251.775,9251.7750,9251.775,9251.775,9251.775,179055.0,179055.000,179055.00,179055.000,179055.0,179055.0,179055.000,179055.00,179055.000,179055.0,0.0,0.00,0.0,0.00,0

### Abspeichern der Daten

In [40]:
previous_application.to_csv(DATASET_DIR / "Datengruppierung" / "previous_application.csv")

In [41]:
todo

[]